# Machine Learning Final Project: Blockudoku Double Deep Q Learning Artificial Convolutional Neural Network
## Group 1, Bot Bot
## Group Members: Mohammed Alnasser, Jesus Nu ̃nez, Ankit Dhingra

In [ ]:
import numpy as np
